In [1]:
!unzip new_age.zip

Streaming output truncated to the last 5000 lines.
  inflating: new_age/train/Train/7588.jpg  
  inflating: new_age/train/Train/7634.jpg  
  inflating: new_age/train/Train/7636.jpg  
  inflating: new_age/train/Train/7656.jpg  
  inflating: new_age/train/Train/7665.jpg  
  inflating: new_age/train/Train/7672.jpg  
  inflating: new_age/train/Train/7676.jpg  
  inflating: new_age/train/Train/7702.jpg  
  inflating: new_age/train/Train/7708.jpg  
  inflating: new_age/train/Train/7713.jpg  
  inflating: new_age/train/Train/7717.jpg  
  inflating: new_age/train/Train/772.jpg  
  inflating: new_age/train/Train/7771.jpg  
  inflating: new_age/train/Train/7790.jpg  
  inflating: new_age/train/Train/780.jpg  
  inflating: new_age/train/Train/7807.jpg  
  inflating: new_age/train/Train/781.jpg  
  inflating: new_age/train/Train/7820.jpg  
  inflating: new_age/train/Train/7830.jpg  
  inflating: new_age/train/Train/7903.jpg  
  inflating: new_age/train/Train/7931.jpg  
  inflating: new_age/train/T

In [5]:
!pip install scipy==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
#import imageio
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
#from scipy.misc import imresize
from skimage.transform import resize 
#from scipy.misc import imread
from imageio import imread
import keras 
import numpy as np

In [8]:
train=pd.read_csv("/content/new_age/train/train.csv")
test=pd.read_csv("/content/new_age/test/test.csv")

In [9]:
train

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE
...,...,...
19901,2482.jpg,MIDDLE
19902,20085.jpg,YOUNG
19903,19663.jpg,MIDDLE
19904,10132.jpg,MIDDLE


In [10]:
id=list(train.ID)

In [11]:
temp=[]
for i in id:
  img=imread("/content/new_age/train/Train/"+str(i))
  img=resize(img,(32,32))
  img=img.astype('float32')
  temp.append(img)
train_x=np.stack(temp)


In [12]:
temp=[]
id=list(test.ID)
for i in id:
  img=imread("/content/new_age/test/Test/"+str(i))
  img=resize(img,(32,32))
  img=img.astype('float32')
  temp.append(img)
test_x=np.stack(temp)


In [13]:
train_x=train_x/255. 
test_x=test_x/255. 

In [14]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.np_utils.to_categorical(train_y)

In [15]:
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 5
batch_size = 128

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer

In [17]:
model = Sequential([
  InputLayer(input_shape=input_num_units),
  Flatten(),
  Dense(units=hidden_num_units, activation='relu'),
  Dense(units=output_num_units, activation='softmax'),
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 500)               1536500   
                                                                 
 dense_1 (Dense)             (None, 3)                 1503      
                                                                 
Total params: 1,538,003
Trainable params: 1,538,003
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/5
156/156 [==============================] - 3s 18ms/step - loss: 1.0256 - accuracy: 0.5428
Epoch 2/5
156/156 [==============================] - 3s 18ms/step - loss: 0.9688 - accuracy: 0.5428
Epoch 3/5
156/156 [==============================] - 3s 18ms/step - loss: 0.9572 - accuracy: 0.5428
Epoch 4/5
156/156 [==============================] - 3s 18ms/step - loss: 0.9541 - accuracy: 0.5428
Epoch 5/5
156/156 [==============================] - 3s 18ms/step - loss: 0.9532 - accuracy: 0.5428


In [20]:
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1, validation_split=0.2)

Epoch 1/5
125/125 [==============================] - 3s 22ms/step - loss: 0.9541 - accuracy: 0.5425 - val_loss: 0.9483 - val_accuracy: 0.5439
Epoch 2/5
125/125 [==============================] - 3s 20ms/step - loss: 0.9540 - accuracy: 0.5425 - val_loss: 0.9481 - val_accuracy: 0.5439
Epoch 3/5
125/125 [==============================] - 3s 20ms/step - loss: 0.9540 - accuracy: 0.5425 - val_loss: 0.9481 - val_accuracy: 0.5439
Epoch 4/5
125/125 [==============================] - 3s 27ms/step - loss: 0.9540 - accuracy: 0.5425 - val_loss: 0.9480 - val_accuracy: 0.5439
Epoch 5/5
125/125 [==============================] - 3s 22ms/step - loss: 0.9540 - accuracy: 0.5425 - val_loss: 0.9480 - val_accuracy: 0.5439


In [23]:
#pred = model.predict_classes(test_x)
#pred = lb.inverse_transform(pred)
#print(pred)
predict_x=model.predict(test_x) 
pred=np.argmax(predict_x,axis=1)
print(pred)
p=lb.inverse_transform(pred)
print(p)
test['Class'] = p
test.to_csv("sub.csv", index=False)

[0 0 0 ... 0 0 0]
['MIDDLE' 'MIDDLE' 'MIDDLE' ... 'MIDDLE' 'MIDDLE' 'MIDDLE']
